# Librerías

In [1]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505213 sha256=ece3c848ec929a9a880ffe920a3e856fab39719dfaa6506149770530793dd8b1
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
# Manejo de Datos
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import sys
import datetime
import random

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Estadísticas y Pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde

# Procesamiento de Datos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Modelado y Algoritmos
from sklearn import neighbors
import joblib
#from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# Interactividad
from ipywidgets import interact

# Google Colab
from google.colab import drive
from google.colab import files

# Otros
from collections import Counter

#Librerias de la API fe Gemini
import google.generativeai as genai #Para API de Gemini
import requests # Se usa para hacer solicitudes HTTP. Permite enviar peticiones POST a las APIs para traducción y generación de texto

In [3]:
from surprise import Reader, Dataset #Libreria para modelos de recomendación

# Conectar con google drive


In [4]:
drive.flush_and_unmount() #Linea en caso de tener que desconectar el drive por algún tipo de falla

Drive not mounted, so nothing to flush and unmount.


In [5]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [6]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [7]:
import a_funciones as fn #Importar el documento de funciones para hacer uso de estas

# Despliegue

In [8]:
def procesar():
    conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies2') #Crear la conexión con la base de datos
    cur = conn.cursor() #Creacion del cursos para realizar consultas dentro del mismo SQL

    fn.ejecutar_sql('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/joins.sql',cur)

    df_final=pd.read_sql("""select * from df_final""",conn)

    ###DUMIZAR VARIABLE GENEROS
    #Separar columna de generos
    df_genres = pd.get_dummies(df_final['genres']).astype(int)
    #Tomar los datos únicos de cada película sin la columna de géneros
    df_movies = df_final.drop('genres', axis=1).drop_duplicates(subset=['user_id', 'movie_id', 'title'])
    #Agregar información de géneros agrupando por movie_id
    genres_by_movie = df_genres.groupby(df_final['movie_id']).max()
    # Unir los dataframes
    df_terminado = df_movies.set_index('movie_id').join(genres_by_movie).reset_index()

    ###CAMBIO DE TIPO DE VARIABLE
    # Convertir la columna 'year_movies' a tipo entero de 64 bits (int64)
    df_terminado['year_movies'] = df_terminado['year_movies'].astype('int64')
    # Convertir la columna 'movie_id' a tipo objeto
    df_terminado['movie_id'] = df_terminado['movie_id'].astype('object')
    # Convertir la columna 'user_id' a tipo objeto
    df_terminado['user_id'] = df_terminado['user_id'].astype('object')

    ###IMPUTAR ATIPICOS
    df_terminado=fn.impute_outliers_with_mean(df_terminado, 'rating')

    ###ESCALAR VARIABLES
    # Seleccion de las variables a escalar
    numcol = []
    for col in df_terminado.columns:
        if df_terminado[col].dtypes == "int64": #Seleccionar solo las variables con valores enteros
            numcol.append(col)
    # Escalamiento de restar el minimo y dividir sobre el rango
    scaler = MinMaxScaler()
    for col in numcol:
        df_terminado[[col]] = scaler.fit_transform(df_terminado[[col]])

    #Eliminar columnas innecesarias
    df_terminado2 = df_terminado.drop(columns=['user_id', 'rating'])
    df_terminado2 = df_terminado2.drop_duplicates(subset=['movie_id', 'title'])

    # Calcular promedio de rating por película
    promedios = df_terminado.groupby(['movie_id', 'title'])['rating'].mean().reset_index()
    promedios.rename(columns={'rating': 'promedio_rating'}, inplace=True)

    # Unir los promedios al dataframe filtrado
    df_catalogo = pd.merge(df_terminado2, promedios, on=['movie_id', 'title'], how='left')

    #Reordenar las columnas
    cols = df_catalogo.columns.tolist()
    cols_reordenadas = ['movie_id', 'title', 'promedio_rating'] + [col for col in cols if col not in ['movie_id', 'title', 'promedio_rating']]
    df_catalogo = df_catalogo[cols_reordenadas]

    return df_final, df_terminado, df_catalogo, conn, cur

df_final,df_terminado,df_catalogo,conn,cur=procesar()
#df_final base de datos dumizada y escalada
#df_terminado base de datos filtrada (eliminacion de columnas)
#df_catalogo base de datos solo con datos de peliculas y columna de promedio ratings por pelicula

In [9]:
def recomendar(uid=5, iid='5',r_ui=''):
    df_final,df_terminado,df_catalogo,conn,cur=procesar()

    modelo=joblib.load('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/modelo_colaborativo.joblib')
    modelo.predict(uid,iid,r_ui='')
    predset = modelo.trainset.build_anti_testset()
    predictions = modelo.test(predset)
    predictions_df = pd.DataFrame(predictions)
    predictions_df['r_ui'].unique() ### promedio de ratings
    predictions_df. sort_values(by='est',ascending=False)

    return predictions_df

In [10]:
predictions_df=recomendar()

In [11]:
def recomendaciones(predictions_df,list_user, n_recomend=5):
    all_recommendations = []

    # Filtrar las predicciones para el usuario y ordenar por la calificación estimada
    for user_id in list_user:
        predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                            sort_values(by="est", ascending=False).head(n_recomend)

        # Seleccionar las columnas necesarias y renombrarlas
        recomendados = predictions_userID[['uid', 'iid', 'r_ui', 'est']]
        recomendados.columns = ['user_id', 'movie_id', 'promedio_rating_real', 'estimacion_rating']

        all_recommendations.append(recomendados)

    # Concatenar todas las recomendaciones
    recomendaciones_df = pd.concat(all_recommendations, ignore_index=True)

    # Guardar las recomendaciones en la base de datos
    recomendaciones_df.to_sql('reco', conn, if_exists="replace", index=False)

    # Realizar la consulta SQL para obtener los títulos de las películas y eliminar duplicados
    recomendaciones_df = pd.read_sql('''SELECT a.*, b.title
                                  FROM reco a
                                  LEFT JOIN df_final b
                                  ON a.movie_id = b.movie_id''', conn)

    # Eliminar filas duplicadas
    recomendaciones_df = recomendaciones_df.drop_duplicates(subset=['movie_id', 'title'])

    return recomendaciones_df

In [12]:
#la lista debe ir de 0 a 600
list_user=[random.randint(0, 600) for _ in range(5)]
recomendados=recomendaciones(predictions_df,list_user)

In [13]:
recomendados

,user_id,movie_id,promedio_rating_real,estimacion_rating,title
0,492,55247,3.579443,4.586571,Into the Wild (2007)
123,492,2324,3.579443,4.568587,Life Is Beautiful (La Vita è bella) (1997)
475,492,898,3.579443,4.555527,"Philadelphia Story, The (1940)"
562,492,1680,3.579443,4.555355,Sliding Doors (1998)
612,492,105504,3.579443,4.536788,Captain Phillips (2013)
696,469,2959,3.579443,4.634337,Fight Club (1999)
1568,469,741,3.579443,4.566481,Ghost in the Shell (Kôkaku kidôtai) (1995)
1622,469,4973,3.579443,4.491078,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ..."
1862,469,1041,3.579443,4.468655,Secrets & Lies (1996)
1873,469,1249,3.579443,4.457307,"Femme Nikita, La (Nikita) (1990)"


In [25]:
recomendados.to_excel('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/recomendados.xlsx', index=False)